In [ ]:
import pandas as pd
import numpy as np

df_players = pd.read_csv('data/players_data-2024_2025.csv')
for i in df_players.columns:
    print(i)

In [8]:
df_players.shape

(2854, 267)

In [ ]:
df_players_light = pd.read_csv('data/players_data_light-2024_2025.csv')
df_players_light.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,Att (GK),Thr,Launch%,AvgLen,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist
0,1,Max Aarons,eng ENG,DF,Bournemouth,eng Premier League,24.0,2000.0,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Max Aarons,eng ENG,"DF,MF",Valencia,es La Liga,24.0,2000.0,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Rodrigo Abajas,es ESP,DF,Valencia,es La Liga,21.0,2003.0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,James Abankwah,ie IRL,"DF,MF",Udinese,it Serie A,20.0,2004.0,6,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Keyliane Abdallah,fr FRA,FW,Marseille,fr Ligue 1,18.0,2006.0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Tìm top players dựa trên performance
df_players = pd.read_csv("data/players_data-2024_2025.csv")

# Top performers by goals
top_scorers = df_players.nlargest(50, "Gls")[["Player", "Squad", "Comp", "Gls", "Ast"]]
print("Top 50 goal scorers:")
print(top_scorers.head(10))

# Top players by assists  
top_assists = df_players.nlargest(50, "Ast")[["Player", "Squad", "Comp", "Gls", "Ast"]]
print("\nTop 50 assists leaders:")
print(top_assists.head(10))

# Top players by market value teams
big_teams = [
    "Manchester City", "Arsenal", "Liverpool", "Chelsea", "Manchester United",
    "Real Madrid", "Barcelona", "Atletico Madrid", 
    "Bayern Munich", "Borussia Dortmund",
    "Juventus", "AC Milan", "Inter", "Napoli",
    "PSG", "Marseille", "Monaco"
]

top_players_big_teams = df_players[df_players["Squad"].isin(big_teams)]
print(f"\nPlayers from big teams: {len(top_players_big_teams)}")

# Kết hợp các criteria
important_players = pd.concat([
    df_players.nlargest(100, "Gls"),
    df_players.nlargest(50, "Ast"), 
    top_players_big_teams
]).drop_duplicates("Player")

print(f"Total important players: {len(important_players)}")
important_player_names = set(important_players["Player"].tolist())

In [ ]:
# v3 football api
import requests
import time
import datetime, json
# Cấu hình API-Football
API_KEY = "56fe75b656667fd06e234aaa4952f3b1"  
BASE_URL = "https://v3.football.api-sports.io"

headers = {
    "x-rapidapi-key": API_KEY,
    "x-rapidapi-host": "v3.football.api-sports.io"
}

def get_api_football_data(player_name: str) -> dict:
    """Lấy injury status + transfers cho player"""
    
    # 1. Tìm player ID
    search_url = f"{BASE_URL}/players"
    params = {"search": player_name}
    response = requests.get(search_url, headers=headers, params=params)
    
    if response.status_code != 200:
        return {"injury_status": {"status": "not_found"}, "transfers": []}
    
    data = response.json()
    if not data.get("response"):
        return {"injury_status": {"status": "not_found"}, "transfers": []}
    
    player_info = data["response"][0]["player"]
    player_id = player_info["id"]
    
    # 2. Lấy injury status
    injury_url = f"{BASE_URL}/injuries"
    injury_params = {"player": player_id}
    injury_response = requests.get(injury_url, headers=headers, params=injury_params)
    
    injury_status = {"status": "healthy"}  # default
    if injury_response.status_code == 200:
        injury_data = injury_response.json()
        if injury_data.get("response"):
            injury_info = injury_data["response"][0]
            injury_status = {
                "status": "injured",
                "type": injury_info.get("player", {}).get("type", "unknown"),
                "reason": injury_info.get("player", {}).get("reason", ""),
                "last_updated": datetime.now().isoformat()
            }
    
    # 3. Lấy transfers (tóm tắt)
    transfers = []  # API này có thể cần trả phí
    
    time.sleep(0.5)  # Rate limit friendly
    return {
        "injury_status": injury_status,
        "transfers": transfers
    }

# Test với 5 players quan trọng
important_test = ["Lionel Messi", "Cristiano Ronaldo", "Kylian Mbappé", "Erling Haaland", "Vinicius Jr"]

for player in important_test:
    print(f"Getting data for {player}...")
    data = get_api_football_data(player)
    print(f"  Injury: {data['injury_status']['status']}")
    print(f"  Transfers: {len(data['transfers'])}")
    time.sleep(1)  # Respect rate limit

## Get Career Stats by Season

In [ ]:

import soccerdata as sd
import json
import pandas as pd

# Lấy dữ liệu 5 mùa gần nhất của 5 giải VĐQG hàng đầu
# soccerdata sẽ cache dữ liệu, lần chạy sau sẽ rất nhanh
seasons = ["23-24", "22-23", "21-22", "20-21", "19-20","18-19", "17-18", "16-17", "15-16", "14-15"]
try:
    fbref = sd.FBref(leagues="Big 5 European Leagues Combined", seasons=seasons)
    print(f"Đang tải dữ liệu stats cho {len(seasons)} mùa từ FBref...")
    
    # Lấy 'standard stats'
    stats_df = fbref.read_player_season_stats(stat_type='standard')

    # Add helper function để access column safely
    def safe_get(row, category, stat, default=0):
        col = (category, stat)
        value = row.get(col, default)
        if pd.isna(value):
            return default
        return value

    # Chuyển đổi DataFrame sang dict theo format yêu cầu
    career_stats_by_name = {}
    # Group theo tên cầu thủ
    for player_name, player_df in stats_df.groupby(level="player"):
        career = []
        # Sắp xếp theo mùa, mùa mới nhất lên trước
        for idx, row in player_df.sort_index(level='season', ascending=False).iterrows():
    # idx[0] = league, idx[1] = season, idx[2] = team, idx[3] = player
    
            season_str = idx[1]  # season từ index level 1 (như '2324')
            season_formatted = f"20{season_str[:2]}-20{season_str[2:]}"
            
            # ✅ FIXED: Sử dụng correct column access pattern
            season_data = {
                "season": season_formatted,
                "club": idx[2],  # team từ index level 2 ✅
                "league": idx[0],  # league từ index level 0 ✅
                "matches": int(safe_get(row, 'Playing Time', 'MP', 0)),
                "starts": int(safe_get(row, 'Playing Time', 'Starts', 0)),
                "minutes": int(safe_get(row, 'Playing Time', 'Min', 0)),
                "goals": int(safe_get(row, 'Performance', 'Gls', 0)),
                "assists": int(safe_get(row, 'Performance', 'Ast', 0)),
                "xg": safe_get(row, 'Expected', 'xG', 0.0),
                "xa": safe_get(row, 'Expected', 'xAG', 0.0),
                "yellow_cards": int(safe_get(row, 'Performance', 'CrdY', 0)),
                "red_cards": int(safe_get(row, 'Performance', 'CrdR', 0)),
            }
            career.append(season_data)
        
        career_stats_by_name[player_name] = career

    print(f"\nĐã xử lý xong career stats cho {len(career_stats_by_name)} cầu thủ.")

    # In thử dữ liệu của một cầu thủ
    if "Harry Kane" in career_stats_by_name:
        print("\nVí dụ career_stats cho 'Harry Kane':")
        print(json.dumps(career_stats_by_name["Harry Kane"], indent=2))

    # Lưu ra file để cell merge sử dụng
    out_path = "data/career_stats.json"
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(career_stats_by_name, f, ensure_ascii=False)
    print(f"\nĐã lưu career stats vào file: {out_path}")

except Exception as e:
    print(f"Đã có lỗi xảy ra khi lấy dữ liệu từ soccerdata: {e}")


## Metadata Placeholder

In [ ]:
import pandas as pd
import json
import unicodedata
from datetime import datetime, timezone
import wikipediaapi
# 1) Load Kaggle players data
df_players = pd.read_csv("data/players_data-2024_2025.csv")


def norm(s: str) -> str:
    s = "" if pd.isna(s) else str(s)
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    return " ".join(s.lower().split())


def make_entity_id(row) -> str:
    name = norm(row.get("Player", "unknown"))
    club = norm(row.get("Squad", "unknown"))
    league = norm(row.get("Comp", "unknown"))
    base = f"{name}_{club}_{league}_2024_2025"
    return f"player_{base.replace(' ', '_')}"


def num(row, col, default=None):
    v = row.get(col, default)
    try:
        if pd.isna(v):
            return default
        return float(v)
    except (TypeError, ValueError):
        return default

def fetch_wiki_summary(player_name: str) -> str | None:
    if not player_name: return None
    wiki = wikipediaapi.Wikipedia(
        user_agent="RAG_Football/1.0 (mqt26094@gmail.com)",
        language="en"
    )
    page = wiki.page(player_name)
    if page.exists():
        return page.summary
    return "Unknown"


entities = []
now_iso = datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")

for _, row in df_players.iterrows():
    name = row.get("Player", "")
    nationality = row.get("Nation", "")
    pos = row.get("Pos", "")
    club = row.get("Squad", "")
    league = row.get("Comp", "")
    biography = fetch_wiki_summary(name)
    # Stats mùa hiện tại từ Kaggle cho 24-25
    season_stats = {
        "matches": num(row, "MP", 0),
        "starts": num(row, "Starts", 0),
        "minutes": num(row, "Min", 0),
        "full_90s": num(row, "90s", 0),
        "goals": num(row, "Gls", 0),
        "assists": num(row, "Ast", 0),
        "goals_plus_assists": num(row, "G+A", 0),
        "non_penalty_goals": num(row, "G-PK", 0),
        "xg": num(row, "xG"),
        "xa": num(row, "xAG"),
        "yellow_cards": num(row, "CrdY", 0),
        "red_cards": num(row, "CrdR", 0),
        # Các trường dưới sẽ được map nếu dùng bản full hoặc nguồn khác
        "shots": None,
        "shots_on_target": None,
        "key_passes": None,
        "progressive_passes": None,
        "progressive_carries": None,
        "tackles": None,
        "interceptions": None,
    }

    # Tạm coi 24-25 là 1 entry trong career_stats
    career_stats = [
        {
            "season": "2024-2025",
            "club": norm(club),
            "league": norm(league),
            **season_stats,
        }
    ]

    entity = {
        "entity_type": "player",
        "entity_id": make_entity_id(row),
        "name": norm(name),

        "identity": {
            "birth_year": None,          # sẽ fill từ Sportmonks/API khác
            "nationality": norm(nationality),
            "position": norm(pos),
            "height_cm": None,
            "weight_kg": None,
            "dominant_foot": None,
        },

        "current_season": "2024-2025",
        "current_club": norm(club),
        "current_league": norm(league),

        # Stats mùa hiện tại (alias tiện dụng cho truy vấn nhanh)
        "season_stats": season_stats,

        # Lịch sử theo mùa (hiện tại mới có 24-25, sau append thêm từ worldfootballR/API)
        "career_stats": career_stats,

        "big5_career_totals": None,
        # Tổng hợp sự nghiệp (tính lại từ career_stats khi đã đủ dữ liệu)
        "career_totals": {
            "total_career_goals": None,
            "total_career_assists": None,
            "total_career_matches": None,
            "total_career_minutes": None,
        },

        "injury_status": {
            "status": "unknown",
            "last_injury_date": None,
            "details": None,
        },

        "biography": biography,   
        "transfers": [],     # sẽ fill từ Transfermarkt/dataset
    
        "sources": ["kaggle"],
        "last_updated": now_iso,
    }

    entities.append(entity)

output_path = "data/players_metadata_placeholder_2024_2025.jsonl"
with open(output_path, "w", encoding="utf-8") as f:
    for e in entities:
        f.write(json.dumps(e, ensure_ascii=False) + "\n")

len(entities)

## Personal players details

In [ ]:
import json
import requests
import time
from collections import defaultdict


WIKIDATA_SPARQL_URL = "https://query.wikidata.org/sparql"

WIKIDATA_HEADERS = {
    "Accept": "application/sparql-results+json",
    "User-Agent": "RAG_Football/1.0 (contact: mqt2604@gmail.com)"
}


WBSEARCH_URL = "https://www.wikidata.org/w/api.php"

def resolve_names_to_qids(names: list[str]) -> dict[str, list[str]]:
    out = {}
    for name in names:
        if not name or not name.strip():
            continue
        params = {
            "action": "wbsearchentities",
            "search": name,
            "language": "en",
            "type": "item",
            "limit": 3,
            "format": "json",
        }
        attempt = 0
        while attempt < 5:
            r = requests.get(
                WBSEARCH_URL, params=params,
                headers={**WIKIDATA_HEADERS, "Accept-Encoding": "gzip"},
                timeout=30
            )
            if r.status_code in (429, 503):
                ra = r.headers.get("Retry-After")
                wait_s = int(ra) if ra and ra.isdigit() else (2 ** attempt + 0.5)
                time.sleep(wait_s); attempt += 1; continue

            r.raise_for_status()
            hits = r.json().get("search", [])
            qids = [h["id"] for h in hits if "id" in h]
            key = " ".join(name.strip().lower().split())
            if qids:
                out[key] = qids
            break
        time.sleep(0.3)  # lịch sự
    return out


def build_props_query_for_qids(qids: list[str]) -> str:
    values = " ".join([f"wd:{qid}" for qid in qids])
    return f"""
    SELECT DISTINCT ?player ?birthDate ?height ?weight ?countryLabel ?positionLabel ?teamLabel WHERE {{
    VALUES ?player {{ {values} }}
    ?player wdt:P31 wd:Q5 .              
    ?player wdt:P106 wd:Q937857 .    # nghề: cầu thủ bóng đá    
    OPTIONAL {{ ?player wdt:P569 ?birthDate. }}
    OPTIONAL {{ ?player wdt:P2048 ?height. }}
    OPTIONAL {{ ?player wdt:P2067 ?weight. }}
    OPTIONAL {{ ?player wdt:P27 ?country. }}
    OPTIONAL {{ ?player wdt:P413 ?position. }}
    OPTIONAL {{ ?player wdt:P54 ?team. }}   # thành viên CLB (cả hiện tại/lịch sử)
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """




def fetch_wikidata_players_batched(names: list[str], qid_batch: int = 80) -> dict:
    # Phase A: name -> QIDs
    qid_map = resolve_names_to_qids(names)

    all_result = {}
    # Phase B: QIDs -> properties
    all_qids = sorted({qid for qids in qid_map.values() for qid in qids})
    for i in range(0, len(all_qids), qid_batch):
        qids = all_qids[i:i+qid_batch]
        query = build_props_query_for_qids(qids)

        # POST + backoff
        attempt = 0
        while attempt < 5:
            try:
                resp = requests.post(
                    WIKIDATA_SPARQL_URL,
                    data={"query": query, "format": "json"},
                    headers={**WIKIDATA_HEADERS, "Accept-Encoding": "gzip"},
                    timeout=60,
                )
                # xử lý throttle
                if resp.status_code in (429, 503):
                    ra = resp.headers.get("Retry-After")
                    wait_s = int(ra) if ra and ra.isdigit() else (2 ** attempt + 0.5)
                    time.sleep(wait_s)
                    attempt += 1
                    continue

                resp.raise_for_status()
                data = resp.json()

                # 1) Gom kết quả theo QID
                qid_to_rows = {}
                qid_teamset = defaultdict(set)  # <— NEW: gom team theo QID

                for row in data.get("results", {}).get("bindings", []):
                    qid = row["player"]["value"].rsplit("/", 1)[-1]
                    rec = {
                        "wikidata_id": qid,
                        "name": row.get("playerLabel", {}).get("value"),
                        "birth_date": row.get("birthDate", {}).get("value"),
                        "height_m": float(row["height"]["value"]) if "height" in row else None,
                        "weight_kg": float(row["weight"]["value"]) if "weight" in row else None,
                        "nationality": row.get("countryLabel", {}).get("value"),
                        "position": row.get("positionLabel", {}).get("value"),
                    }
                    team = row.get("teamLabel", {}).get("value")
                    if team:
                        qid_teamset[qid].add(team)  # <— gom vào set theo QID

                    qid_to_rows.setdefault(qid, []).append(rec)

                # chuẩn hoá: gán danh sách team đầy đủ cho mọi rec của cùng QID
                for _qid, _rows in qid_to_rows.items():
                    teams_list = sorted(qid_teamset[_qid]) if _qid in qid_teamset else []
                    for _r in _rows:
                        _r["teams"] = teams_list

                # 2) Map về tên gốc + CHỐNG TRÙNG
                for name_key, qids in qid_map.items():
                    bucket = all_result.setdefault(name_key, [])
                    seen = set((r["wikidata_id"],
                                r.get("birth_date"),
                                r.get("height_m"),
                                r.get("weight_kg"),
                                r.get("nationality"),
                                r.get("position")) for r in bucket)
                    for q in qids:
                        for rec in qid_to_rows.get(q, []):
                            sig = (rec["wikidata_id"],
                                rec.get("birth_date"),
                                rec.get("height_m"),
                                rec.get("weight_kg"),
                                rec.get("nationality"),
                                rec.get("position"))
                            if sig in seen:
                                continue
                            seen.add(sig)
                            bucket.append(rec)



                # lịch sự với WDQS: ~ <= 1 req/giây
                time.sleep(1.0)
                break
            except requests.exceptions.Timeout:
                attempt += 1
                time.sleep(2 ** attempt)
            except Exception as e:
                attempt += 1
                time.sleep(1.0)
        # next batch
    return all_result


def meters_to_cm(m):
    if m is None:
        return None
    try:
        return int(round(float(m) * 100))
    except Exception:
        return None

NAT_MAP = {
    "eng": "england", "sco": "scotland", "wal": "wales", "nir": "northern ireland",
    "usa": "united states", "kor": "south korea", "civ": "ivory coast",
    "ned": "netherlands", "ger": "germany", "fra": "france", "esp": "spain",
    "por": "portugal", "bra": "brazil", "arg": "argentina", "uru": "uruguay",
}

def norm_nat(s: str | None) -> str:
    s = _norm(s)
    if not s: return ""
    tok = s.split()[-1]              # FBref/Kaggle đôi khi có 'eng ENG' -> lấy 'eng'
    return NAT_MAP.get(tok, s)

POS_MAP = {
    "gk": {"goalkeeper"},
    "df": {"defender", "centre-back", "center back", "full-back", "wing-back"},
    "mf": {"midfielder", "defensive midfielder", "attacking midfielder", "winger"},
    "fw": {"forward", "striker", "centre-forward", "center forward", "winger"},
}

def pos_match(ent_pos: str, wd_pos: str) -> bool:
    ep = _norm(ent_pos)
    wp = _norm(wd_pos)
    if not ep or not wp: return False
    codes = [t for t in ep.replace("/", ",").split(",") if t]
    for code in codes:
        code = code.strip()
        vocab = POS_MAP.get(code, {code})
        if any(v in wp for v in vocab):
            return True
    return False


def _norm(s: str | None) -> str:
    return " ".join(str(s).strip().lower().split()) if s else ""

def pick_wikidata_candidate(entity: dict, candidates: list[dict]) -> dict | None:
    ident = entity.get("identity", {}) or {}
    ent_nat = norm_nat(ident.get("nationality"))
    ent_pos = _norm(ident.get("position"))

    best = None
    best_score = -1

    for c in candidates:
        score = 0
        c_nat = _norm(c.get("nationality"))
        c_pos = _norm(c.get("position"))

        # match quốc tịch (nới lỏng: uk vs england/wales/scotland/nir)
        if ent_nat and c_nat:
            if ent_nat in c_nat or c_nat in ent_nat:
                score += 1
            elif c_nat == "united kingdom" and ent_nat in {"england","scotland","wales","northern ireland"}:
                score += 1

        # match vị trí (map 'fw/df/mf/gk' sang nhãn Wikidata)
        if ent_pos and c_pos and pos_match(ent_pos, c_pos):
            score += 1

        # ưu tiên có ngày sinh (ổn định hơn)
        if c.get("birth_date"):
            score += 0.2

        if score > best_score:
            best_score = score
            best = c

    # chấp nhận nếu có ít nhất 1 tiêu chí khớp
    return best if best_score >= 1 else None


def enrich_metadata_with_wikidata(metadata_entities: list[dict]) -> list[dict]:
    # 1) Query candidates theo unique name (batch)
    unique_names = sorted({e["name"] for e in metadata_entities if e.get("name")})
    print(f"Querying Wikidata for {len(unique_names)} unique names...")
    wikidata_map = fetch_wikidata_players_batched(unique_names, qid_batch=80)
    print(f"Wikidata returned labels for {len(wikidata_map)} names")

    enriched = 0
    ambiguous = 0

    # 2) Duyệt từng entity (1–1 theo entity_id)
    for e in metadata_entities:
        name_key = _norm(e.get("name"))
        if not name_key:
            continue

        candidates = wikidata_map.get(name_key)
        if not candidates:
            continue

        wd = pick_wikidata_candidate(e, candidates)
        if not wd:
            ambiguous += 1
            continue

        ident = e.setdefault("identity", {})

        # Fill fields nếu còn trống
        if wd.get("birth_date") and not ident.get("birth_year"):
            try:
                ident["birth_year"] = int(wd["birth_date"][:4])
            except Exception:
                pass

        h_cm = meters_to_cm(wd.get("height_m"))
        if h_cm and not ident.get("height_cm"):
            ident["height_cm"] = h_cm

        if wd.get("weight_kg") and not ident.get("weight_kg"):
            try:
                ident["weight_kg"] = int(round(float(wd["weight_kg"])))
            except Exception:
                pass

        wd_nat = wd.get("nationality")
        if wd_nat and not ident.get("nationality"):
            ident["nationality"] = wd_nat

        wd_pos = wd.get("position")
        if wd_pos and not ident.get("position"):
            ident["position"] = _norm(wd_pos)

        # Gắn wikidata_id riêng cho entity này
        e.setdefault("external_ids", {})["wikidata"] = wd["wikidata_id"]
        e.setdefault("sources", [])
        if "wikidata" not in e["sources"]:
            e["sources"].append("wikidata")

        enriched += 1

    print(f"Enriched {enriched} / {len(metadata_entities)} entities with Wikidata")
    print(f"Skipped {ambiguous} entities due to ambiguous or low-confidence matches")
    return metadata_entities


metadata_entities = []
with open("/content/drive/MyDrive/players_metadata_placeholder_2024_2025.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        metadata_entities.append(json.loads(line))

metadata_entities = enrich_metadata_with_wikidata(metadata_entities)

output_path_wd = "/content/drive/MyDrive/players_metadata_wikidata_2024_2025.jsonl"
with open(output_path_wd, "w", encoding="utf-8") as f:
    for e in metadata_entities:
        f.write(json.dumps(e, ensure_ascii=False) + "\n")

print(f"Saved Wikidata-enriched metadata to {output_path_wd}")

## merge career with available metadata

In [3]:
# Merge career stats với metadata placeholder
import json
import pandas as pd
from datetime import datetime

# Load career stats
with open("data/players/career_stats.json", "r", encoding="utf-8") as f:
    career_stats = json.load(f)

metadata_entities = []
with open("data/players/players_metadata_wikidata_2024_2025.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            metadata_entities.append(json.loads(line))

print(f"Loaded {len(career_stats)} career stats")
print(f"Loaded {len(metadata_entities)} metadata entities")

# Merge function
def merge_career_with_metadata(career_stats, metadata_entities):
    merged = []
    
    # Create lookup dict for career stats
    career_lookup = {}
    for player_name, career_data in career_stats.items():
        norm_name = player_name.lower().strip()
        career_lookup[norm_name] = career_data
    
    # Merge with metadata
    for entity in metadata_entities:
        player_name = entity["name"]
        norm_name = player_name.lower().strip()
        
        if norm_name in career_lookup:
            # Merge career stats
            entity["career_stats"] = career_lookup[norm_name]
            
            # Update career totals
            total_goals = sum(season["goals"] for season in career_lookup[norm_name])
            total_assists = sum(season["assists"] for season in career_lookup[norm_name])
            total_matches = sum(season["matches"] for season in career_lookup[norm_name])
            total_minutes = sum(season["minutes"] for season in career_lookup[norm_name])
            
            entity["big5_career_totals"] = {
                "total_big5_goals": total_goals,
                "total_big5_assists": total_assists,
                "total_big5_matches": total_matches,
                "total_big5_minutes": total_minutes,
                "coverage": "Big 5 European Leagues (2014-2015 to 2024-2025)",
                "note": "Chỉ bao gồm thời gian chơi trong Big 5 leagues, không phải toàn bộ sự nghiệp"
                }
            
            
            entity["sources"].append("soccerdata")
        
        merged.append(entity)
    
    return merged

# Execute merge
merged_entities = merge_career_with_metadata(career_stats, metadata_entities)

# Save merged data
output_path = "data/players/players_complete_metadata.jsonl"
with open(output_path, "w", encoding="utf-8") as f:
    for entity in merged_entities:
        f.write(json.dumps(entity, ensure_ascii=False) + "\n")

print(f"\n✅ Merged {len(merged_entities)} complete player entities")
print(f"📁 Saved to: {output_path}")

# Test với Harry Kane
kane_entity = next((e for e in merged_entities if "harry kane" in e["name"]), None)
if kane_entity:
    print(f"\n🎯 Harry Kane Big 5 career totals:")
    print(f"  Goals: {kane_entity['big5_career_totals']['total_big5_goals']}")
    print(f"  Assists: {kane_entity['big5_career_totals']['total_big5_assists']}")
    print(f"  Matches: {kane_entity['big5_career_totals']['total_big5_matches']}")
    
    # Show career_totals vẫn là placeholder
    print(f"\n🎯 Harry Kane complete career totals (placeholder):")
    print(f"  Goals: {kane_entity['career_totals']['total_career_goals']} (chưa có data)")
    print(f"  Assists: {kane_entity['career_totals']['total_career_assists']} (chưa có data)")

Loaded 8211 career stats
Loaded 2854 metadata entities

✅ Merged 2854 complete player entities
📁 Saved to: data/players/players_complete_metadata.jsonl

🎯 Harry Kane Big 5 career totals:
  Goals: 246
  Assists: 52
  Matches: 338

🎯 Harry Kane complete career totals (placeholder):
  Goals: None (chưa có data)
  Assists: None (chưa có data)
